In [3]:
import db
import pickle
import joblib
import numpy as np
from random import choice, choices, randint, randrange, random, sample, shuffle
from sklearn.utils import shuffle
from keras.models import load_model

dbConn = db.DataBase()
model_dnn = load_model('./models/dnn.h5')
# model_rf = joblib.load("./models/rf.joblib")
gods = dbConn.select("SELECT * FROM God ORDER BY name", ())
matches = dbConn.select("SELECT "
                        "t1_god1, t1_god2, t1_god3, t1_god4, t1_god5, "
                        "t2_god1, t2_god2, t2_god3, t2_god4, t2_god5, "
                        "ban1, ban2, ban3, ban4, ban5, ban6, ban7, ban8, ban9, ban10, "
                        "win FROM Match", ())

In [4]:
def get_win_good_with():
    # synergy win rate
    gods_ids = list(map(lambda x: x[0], gods))
    good_with = np.zeros((len(gods_ids), len(gods_ids))) * 0.5
    good_with_num_matches = np.zeros((len(gods_ids), len(gods_ids)))
    for match in matches:
        for god_1_id in match[:5]:
            i = gods_ids.index(god_1_id)
            for god_2_id in match[:5]:
                if god_1_id != god_2_id:
                    j = gods_ids.index(god_2_id)
                    good_with[i, j] += 1 if match[len(match) - 1] == 1 else 0
                    good_with_num_matches[i, j] += 1

        for god_1_id in match[5:10]:
            i = gods_ids.index(god_1_id)
            for god_2_id in match[5:10]:
                if god_1_id != god_2_id:
                    j = gods_ids.index(god_2_id)
                    good_with[i, j] += 1 if match[len(match) - 1] == 2 else 0
                    good_with_num_matches[i, j] += 1

    win_good_with = np.divide(good_with, good_with_num_matches, out=np.zeros_like(good_with), where=good_with_num_matches != 0)
    pickle.dump(win_good_with, open("./pickles/win_good_with.p", "wb"))

    return win_good_with


def get_win_good_against():
    # countering win rate
    gods_ids = list(map(lambda x: x[0], gods))
    good_against = np.zeros((len(gods_ids), len(gods_ids))) * 0.5
    good_against_num_matches = np.zeros((len(gods_ids), len(gods_ids)))
    for match in matches:
        for god_1_id in match[:5]:
            i = gods_ids.index(god_1_id)
            for god_2_id in match[5:10]:
                j = gods_ids.index(god_2_id)

                if match[len(match) - 1] == 1:
                    good_against[i, j] += 1
                else:
                    good_against[j, i] += 1

                good_against_num_matches[i, j] += 1
                good_against_num_matches[j, i] += 1

    win_good_against = np.divide(good_against, good_against_num_matches, out=np.zeros_like(good_against), where=good_against_num_matches != 0)
    pickle.dump(win_good_against, open("./pickles/win_good_against.p", "wb"))

    return win_good_against

win_good_with = get_win_good_with()
win_good_against = get_win_good_against()

In [5]:
def get_synergy_counter(gods_team_1, gods_team_2):
    team_1 = []
    team_2 = []
    for i in range(len(gods_team_1)):
        if gods_team_1[i] == 1:
            team_1.append(gods[i][1])
    for i in range(len(gods_team_2)):
        if gods_team_2[i] == 1:
            team_2.append(gods[i][1])
            
    if len(team_1) < 2 or len(team_2) < 2:
        return 0, 0
    
    gods_names = list(map(lambda x: x[1], gods))
    s_team_1 = 0
    s_team_2 = 0
    n_synergy_team_1 = 0
    n_synergy_team_2 = 0
    counter = 0
    n_counter = 0
    for god_1_name in team_1:
        i = gods_names.index(god_1_name)
        for god_2_name in team_1:
            if god_1_name != god_2_name:
                j = gods_names.index(god_2_name)
                s_team_1 += win_good_with[i, j] if win_good_with[i, j] > 0 else 0.5
                n_synergy_team_1 += 1

    for god_1_name in team_2:
        i = gods_names.index(god_1_name)
        for god_2_name in team_2:
            if god_1_name != god_2_name:
                j = gods_names.index(god_2_name)
                s_team_2 += win_good_with[i, j] if win_good_with[i, j] > 0 else 0.5
                n_synergy_team_2 += 1

        for god_2_name in team_1:
            if god_1_name != god_2_name:
                j = gods_names.index(god_2_name)
                counter += win_good_against[i, j]
                n_counter += 1

    s_team_1 = s_team_1 / n_synergy_team_1
    s_team_2 = s_team_2 / n_synergy_team_2
    synergy = s_team_1 / (s_team_1 + s_team_2)
    return synergy, counter / n_counter

def get_god_selection(team_1, team_2):
    gods_team_1 = list(map(lambda god: 1 if god[1] in team_1 else 0, gods))
    gods_team_2 = list(map(lambda god: 1 if god[1] in team_2 else 0, gods))
    return gods_team_1, gods_team_2

def get_god_selection_names(gods_selection):
    teams = np.split(gods_selection, 2)
    
    team_1 = []
    team_2 = []
    for i in range(len(teams[0])):
        if teams[0][i] == 1:
            team_1.append(gods[i][1])
    for i in range(len(teams[1])):
        if teams[1][i] == 1:
            team_2.append(gods[i][1])
            
    return team_1, team_2

def predict(features):
    # make win probability prediction with the model
    
    # DNN
    prediction = model_dnn.predict([features])
    
    # random forest
    # prediction = model_rf.predict([features])[0]
    
    #print(prediction)
    return prediction

In [13]:
def generate_genome(preselection, enemy_team):
    # generate ally team equal to preselection
    team = np.copy(preselection)

    # generate a list of possible character choices avoiding duplicates
    preselection_ones = np.where(preselection == 1)[0].tolist()
    enemy_team_ones = np.where(enemy_team == 1)[0].tolist()
    possible_ones = np.setdiff1d(np.arange(0, len(preselection)), preselection_ones)
    possible_ones = np.setdiff1d(possible_ones, enemy_team_ones)

    # randomly select characters to fill the ally team
    chosen_ones = sample([*possible_ones], k=5-len(preselection_ones))
    team[chosen_ones[0:5-len(preselection_ones)]] = 1

    # return ally team as onehot encode
    return team

def generate_population(preselection, enemy_team, size=10):
    # generate `size` genomes in a population
    return [generate_genome(preselection, enemy_team) for _ in range(size)]

def fitness(team, enemy_team):
    # calculate synergy and counter features for given genome
    genome = np.append(team, enemy_team)
    synergy, counter = get_synergy_counter(team.tolist(), enemy_team.tolist())
    genome = np.append(genome, synergy)
    genome = np.append(genome, counter)
    
    # use prediction model to get win probability for given character selection
    return predict(genome.tolist())

def selection_pair(population, fitness_func, enemy_team, team_i):
    # randomly select 2 genomes from the population weighted by genomes fitness score
    return choices(
        population=population,
        weights=[fitness(genome, enemy_team) if team_i == 1 else 1 - fitness(genome, enemy_team) for genome in population],
        k=2
    )

def single_point_crossover(a, b, team_preselection, enemy_team):
    # calculate indices of ones
    preselection_ones = np.where(team_preselection == 1)[0].tolist()
    enemy_team_ones = np.where(enemy_team == 1)[0].tolist()
    a_ones = np.where(a == 1)[0].tolist()
    b_ones = np.where(b == 1)[0].tolist()
    
    # define ones that can be changed
    a_ones_changable = np.setdiff1d(a_ones, preselection_ones)
    a_ones_changable = np.setdiff1d(a_ones_changable, enemy_team_ones)
    b_ones_changable = np.setdiff1d(a_ones, preselection_ones)
    b_ones_changable = np.setdiff1d(b_ones_changable, enemy_team_ones)
    
    # calculate ones unique to genome a and ones unique to genome b that can be changed
    a_only_ones = shuffle(list(set(a_ones_changable) - set(b_ones_changable)))
    b_only_ones = shuffle(list(set(b_ones_changable) - set(a_ones_changable)))
    
    # if 1st team from genome a has any different ones than 1st team from genome b execute crossover
    if len(a_only_ones) > 0:
        
        # calculate random number of ones to switch
        max_num_ones_changable = len(a_only_ones) if len(a_only_ones) < len(b_only_ones) else len(b_only_ones)
        p = randint(1, max_num_ones_changable)
        
        # switch 0 and 1 in 1st teams of genomes
        for i in range(p):
            a[a_only_ones[i]] = 0
            b[a_only_ones[i]] = 1

            a[b_only_ones[i]] = 1
            b[b_only_ones[i]] = 0
    
    # return new genomes
    return a, b

def mutation(a, team_preselection, enemy_team, num=1):
    # calculate indices of ones and zeros
    preselection_ones = np.where(team_preselection == 1)[0].tolist()
    enemy_team_ones = np.where(enemy_team == 1)[0].tolist()
    ones = np.where(a == 1)[0].tolist()
    zeros = np.where(a == 0)[0].tolist()
    
    # define ones and zeros that can be changed
    ones_changable = np.setdiff1d(ones, preselection_ones)
    zeros_changable = np.setdiff1d(zeros, enemy_team_ones)

    # execute `num` mutations on 1st team
    max_num_ones_changable = num if len(ones_changable) > num else len(ones_changable)
    for _ in range(max_num_ones_changable):

        # change random 1 into 0
        index = choice(ones_changable)
        a[index] = 0

        # change random 0 into 1
        index = choice(zeros_changable)
        a[index] = 1
        
    # return mutated genome
    return a

In [25]:
def run_evolution(populate_func,
                  fitness_func,
                  fitness_limit,
                  selection_func,
                  crossover_func,
                  mutation_func,
                  preselection_team_1,
                  preselection_team_2,
                  characters_num=121,
                  generation_limit=100):
    
    # initialize random best combinations
    current_best_team_1 = generate_genome(preselection_team_1, np.zeros(characters_num))
    current_best_team_2 = generate_genome(preselection_team_2, np.zeros(characters_num))
    
    # generate population
    population_1 = populate_func(preselection_team_1, current_best_team_2)
    population_2 = populate_func(preselection_team_2, current_best_team_1)
    
    # generations loop, evolve teams alternately (1st gen evolve team 2, 2nd gen evolve team 1, 3rd gen evolve team 2, etc.)
    for i in range(generation_limit):
                
        # break the loop if top genome already reached fitness limit
        # if fitness_func(population[0]) <= fitness_limit:
        #     break

        if i % 2 == 0:
            # evolve team 2
            # sort population by fitness descending, closer to 1 means team 2 wins
            population_2 = sorted(
                population_2,
                key=lambda genome: fitness(genome, current_best_team_1),
                reverse=True
            )
            
            # add top 2 genomes into the next generation
            next_generation = population_2[0:2]

            # repeat for half of number of genomes in population, to create an equally large new generation
            for j in range(int(len(population_2) / 2) - 1):

                # select parents and create offspring
                parents = selection_func(population_2, fitness_func, current_best_team_1, 1)
                offspring_a, offspring_b = crossover_func(parents[0], parents[1], preselection_team_2, current_best_team_1)
                offspring_a = mutation_func(offspring_a, preselection_team_2, current_best_team_1)
                offspring_b = mutation_func(offspring_b, preselection_team_2, current_best_team_1)

                # add their offsprings into the next generation
                next_generation += [offspring_a, offspring_b]
                
            # set current best team 2 to counter current best team 1
            population_2 = sorted(
                population_2,
                key=lambda genome: fitness(genome, current_best_team_1),
                reverse=True
            )
            current_best_team_2 = population_2[0]
            print(f"team 2 generation's best: {fitness_func(current_best_team_2, current_best_team_1)[0][0]}")
            print(f"{get_god_selection_names(np.append(current_best_team_1, current_best_team_2))}")
        else:
            # evolve team 1
            # sort population by fitness ascending, closer to 0 means team 1 wins
            population_1 = sorted(
                population_1,
                key=lambda genome: fitness(genome, current_best_team_2)
            )

            # add top 2 genomes into the next generation
            next_generation = population_1[0:2]

            # repeat for half of number of genomes in population, to create an equally large new generation
            for j in range(int(len(population_1) / 2) - 1):

                # select parents and create offspring
                parents = selection_func(population_1, fitness_func, current_best_team_2, 0)
                offspring_a, offspring_b = crossover_func(parents[0], parents[1], preselection_team_1, current_best_team_2)
                offspring_a = mutation_func(offspring_a, preselection_team_1, current_best_team_2)
                offspring_b = mutation_func(offspring_b, preselection_team_1, current_best_team_2)

                # add their offsprings into the next generation
                next_generation += [offspring_a, offspring_b]

            # set current best team 1 to counter current best team 2
            population_1 = sorted(
                population_1,
                key=lambda genome: fitness(genome, current_best_team_2)
            )
            current_best_team_1 = population_1[0]
            print(f"team 1 generation's best: {fitness_func(current_best_team_1, current_best_team_2)[0][0]}")
            print(f"{get_god_selection_names(np.append(current_best_team_1, current_best_team_2))}")

        population = next_generation
            
    # sort population by fitness ascending
    population_1 = sorted(
        population_1,
        key=lambda genome: fitness(genome, current_best_team_2)
    )

    # return population and number of generations
    return population_1[0], population_2[0], i, fitness_func(population_1[0], current_best_team_2)

In [29]:
preteam_1, preteam_2 = get_god_selection(['Achilles', 'Ares'],
                                         ['Agni', 'Athena'])

team_1, team_2, generations, best_win_prob = run_evolution(
    populate_func=generate_population,
    fitness_func=fitness,
    fitness_limit=0.3,
    selection_func=selection_pair,
    crossover_func=single_point_crossover,
    mutation_func=mutation,
    preselection_team_1=np.array(preteam_1),
    preselection_team_2=np.array(preteam_2),
    generation_limit=100
)

print(f"number of generations: {generations}")
print(f"best solution: {get_god_selection_names(np.append(team_1, team_2))}")
print(f"best solution win probability: {best_win_prob}")

team 2 generation's best: 0.49046263098716736
(['Achilles', 'Ares', 'Camazotz', 'Pele', 'Zeus'], ['Agni', 'Athena', 'Cabrakan', 'Jing Wei', 'Kuzenbo'])
team 1 generation's best: 0.26087647676467896
(['Achilles', 'Ares', 'Bakasura', 'Janus', 'Ymir'], ['Agni', 'Athena', 'Cabrakan', 'Jing Wei', 'Kuzenbo'])
team 2 generation's best: 0.4088708460330963
(['Achilles', 'Ares', 'Bakasura', 'Janus', 'Ymir'], ['Agni', 'Athena', 'Bacchus', 'Chaac', 'Sylvanus'])
team 1 generation's best: 0.31051409244537354
(['Achilles', 'Ao Kuang', 'Ares', 'Kali', 'Raijin'], ['Agni', 'Athena', 'Bacchus', 'Chaac', 'Sylvanus'])
team 2 generation's best: 0.9255102276802063
(['Achilles', 'Ao Kuang', 'Ares', 'Kali', 'Raijin'], ['Agni', 'Athena', 'Freya', 'Ra', 'Scylla'])
team 1 generation's best: 0.03378942608833313
(['Achilles', 'Amaterasu', 'Ares', 'Baba Yaga', 'Hades'], ['Agni', 'Athena', 'Freya', 'Ra', 'Scylla'])
team 2 generation's best: 0.7683461904525757
(['Achilles', 'Amaterasu', 'Ares', 'Baba Yaga', 'Hades'], 

team 2 generation's best: 0.26622891426086426
(['Achilles', 'Ares', 'Hachiman', 'Serqet', 'Thor'], ['Agni', 'Athena', 'Chiron', 'Da Ji', 'Freya'])
team 1 generation's best: 0.3688872456550598
(['Achilles', 'Ares', 'Artemis', 'Heimdallr', 'Tsukuyomi'], ['Agni', 'Athena', 'Chiron', 'Da Ji', 'Freya'])
team 2 generation's best: 0.5715456008911133
(['Achilles', 'Ares', 'Artemis', 'Heimdallr', 'Tsukuyomi'], ['Agni', 'Athena', 'Hun Batz', 'Khepri', 'Ra'])
team 1 generation's best: 0.04931184649467468
(['Achilles', 'Ares', 'Hachiman', 'Hou Yi', 'Thanatos'], ['Agni', 'Athena', 'Hun Batz', 'Khepri', 'Ra'])
team 2 generation's best: 0.9461561441421509
(['Achilles', 'Ares', 'Hachiman', 'Hou Yi', 'Thanatos'], ['Agni', 'Ah Puch', 'Athena', 'Heimdallr', 'Nox'])
team 1 generation's best: 0.046194106340408325
(['Achilles', 'Ares', 'Hachiman', 'Hou Yi', 'Thanatos'], ['Agni', 'Ah Puch', 'Athena', 'Heimdallr', 'Nox'])
team 2 generation's best: 0.9371369481086731
(['Achilles', 'Ares', 'Hachiman', 'Hou Yi',